# Distributed Homomorphic Public Key Encryption

In [329]:
from math import gcd
from random import randint, choice
from sympy import discrete_log
n = 16

## Generate group Z*p

In [330]:
def is_prime(n: int):
    for i in range(2, n // 2):
        if n % i == 0:
            return False
    return True

assert is_prime(17)
assert is_prime(47)
assert not is_prime(16)

In [331]:
class Group:
    def __generate(self):
        p, g = self.p, self.g
        l = []
        cur = g
        while cur != 1:
            l.append(cur)
            cur = (cur * g) % p
        l.append(1)
        self.q = len(l)
        self.group = tuple(l)

    def __init__(self, p, g):
        assert is_prime(p), f"Value for p {p} is not a prime"
        assert g < p, f"Equation g < p is not satisfied when g = {g} and p = {p}"
        self.p = p
        self.g = g
        self.__generate()

    def __str__(self):
        return f"(p: {self.p}, g: {self.g}, q: {self.q}): G = {self.group}"

    def getValues(self):
        return self.p, self.g, self.q, self.group

    def getP(self):
        return self.p

    def getG(self):
        return self.g

    def getQ(self):
        return self.q

    def getGroup(self):
        return self.group
    
    def isMember(self, x: int):
        return x in self.group
    
    def getInverse(self, x: int):
        p = self.getP()
        return pow(x, p - 2, p)
    
    def getRandomGroupFromSameOrder(self):
        p, q, g = self.p, self.q, self.g
        r = randint(1, q - 1)
        while gcd(q, r) != 1:
            r = randint(1, q - 1)
        gp = pow(g, r, p)
        newGroup = Group(p, gp)
        assert self.getQ() == newGroup.getQ(), f"{self.getQ()} vs {newGroup.getQ()}"
        return newGroup

group = Group(47, 17)
print(group)
assert group.getGroup() == (17, 7, 25, 2, 34, 14, 3, 4, 21, 28, 6, 8, 42, 9, 12, 16, 37, 18, 24, 32, 27, 36, 1)
assert group.isMember(17)
assert group.isMember(1)
assert group.isMember(42)
assert not group.isMember(15)
for each in group.getGroup():
    assert (each * group.getInverse(each)) % group.getP() == 1

(p: 47, g: 17, q: 23): G = (17, 7, 25, 2, 34, 14, 3, 4, 21, 28, 6, 8, 42, 9, 12, 16, 37, 18, 24, 32, 27, 36, 1)


In [332]:
group = Group(43, 19)

## Diffie Hellman

In [333]:
class DiffieHellmanParty:
    def __init__(self, group: Group):
        self.group = group
        self.r = None
        self.secret = None

    def __str__(self):
        return f"Secret: {self.secret}, r: {self.r}"

    def getSecret(self):
        return self.secret

    def query(self):
        p, g, q, _ = self.group.getValues()
        r = randint(0, q - 1)
        self.r = r
        token = pow(g, r, p)
        return token

    def reply(self, token: int):
        p, g, q, _ = self.group.getValues()
        r = randint(0, q - 1)
        self.r = r
        self.process(token)
        newToken = pow(g, r, p)
        return newToken
    
    def process(self, token: int):
        p = self.group.getP()
        r = self.r
        self.secret = pow(token, r, p)

X = DiffieHellmanParty(group)
Y = DiffieHellmanParty(group)
A = X.query()
B = Y.reply(A)
X.process(B)
print(f"A = {A}, B = {B}, Xsecret = {X.getSecret()}, Ysecret = {Y.getSecret()}")

A = 23, B = 13, Xsecret = 17, Ysecret = 17


In [334]:
def diffie_hellman_transcript(X: DiffieHellmanParty, Y: DiffieHellmanParty):
    for i in range(1000): # Only A and B are visible in the wild
        A = X.query()
        B = Y.reply(A)
        X.process(B)
        assert X.getSecret() != None
        assert Y.getSecret() != None
        assert X.getSecret() == Y.getSecret()

diffie_hellman_transcript(X, Y)

## ElGamal Encryption

In [335]:
class ElGamalServer:
    def __init__(self, group: Group):
        self.group = group
        self.generatePublicKey()
    
    def __str__(self):
        return f"sk = {self.x}, pk = {self.h}"
    
    def generatePublicKey(self):
        p, g, q, _ = self.group.getValues()
        x = randint(0, q - 1)
        self.x = x
        h = pow(g, x, p)
        self.h = h
    
    def getPublicKey(self):
        return self.h
    
    def decrypt(self, c1: int, c2: int):
        p, g, q, _ = self.group.getValues()
        x = self.x
        tmp = pow(c1, x, p)
        tmp_1 = self.group.getInverse(tmp)
        m = (c2 * tmp_1) % p
        return m
    
class ElGamalClient:
    def __init__(self, group: Group):
        self.group = group
        self.generateSecret()

    def __str__(self):
        return f"secret = {self.y}"
    
    def generateSecret(self):
        q = self.group.getQ()
        y = randint(0, q - 1)
        self.y = y

    def encrypt(self, h: int, m: int):
        p, g, q, _ = self.group.getValues()
        assert 1 <= m < q
        y = self.y
        c1 = pow(g, y, p)
        c2 = (m * pow(h, y, p)) % p
        return c1, c2
    
server = ElGamalServer(group)
client = ElGamalClient(group)
"""
Value 0 is banished because it is not secure as m = 0 => c2 = 0
so trivial for malicious user to find it (reason for what we use exponential ElGamal)
"""
pk = server.getPublicKey()
m = randint(1, group.getQ() - 1)
enc_m = client.encrypt(pk, m)
dec_m = server.decrypt(enc_m[0], enc_m[1])
print(f"pk = {pk}, m = {m}, enc_m = {enc_m}, dec_m = {dec_m}")
assert m == dec_m

pk = 32, m = 7, enc_m = (37, 36), dec_m = 7


In [336]:
def transcript_ElGamal(server: ElGamalServer, client: ElGamalClient):
    for i in range(100):
        server.generatePublicKey()
        for j in range(100):
            client.generateSecret()
            pk = server.getPublicKey()
            m = randint(1, group.getQ() - 1)
            enc_m = client.encrypt(pk, m)
            dec_m = server.decrypt(enc_m[0], enc_m[1])
            assert m == dec_m

transcript_ElGamal(server, client)

## Exponential ElGamal Encryption

In [337]:
class ExponentialElGamalServer(ElGamalServer):
    def __init__(self, group: Group):
        super().__init__(group)
        self.reset()

    def reset(self):
        self.c1p = 1
        self.c2p = 1

    def receive(self, c1: int, c2: int):
        self.c1p *= c1
        self.c2p *= c2

    def getTotal(self):
        c1p = self.c1p
        c2p = self.c2p
        mp = super().decrypt(c1p, c2p)
        p, g, q, _ = self.group.getValues()
        return discrete_log(p, mp, g) # If the sum of votes 'm' is small it is feasible in a reasonable time

class ExponentialElGamalTrustee(ElGamalClient):
    def __init__(self, group: Group):
        super().__init__(group)

    def vote(self, h: int, m: int):
        assert m == 0 or m == 1
        p, g, q, _ = self.group.getValues()
        mp = pow(g, m, p)
        return super().encrypt(h, mp)
    
agency = ExponentialElGamalServer(group)
pk = agency.getPublicKey()

t0 = ExponentialElGamalTrustee(group)
t1 = ExponentialElGamalTrustee(group)
t2 = ExponentialElGamalTrustee(group)

enc_v0 = t0.vote(pk, 1)
enc_v1 = t1.vote(pk, 0)
enc_v2 = t2.vote(pk, 1)

agency.receive(enc_v0[0], enc_v0[1])
agency.receive(enc_v1[0], enc_v1[1])
agency.receive(enc_v2[0], enc_v2[1])

print(t0, t1, t2)
print(agency)
print(f"enc_v0 = {enc_v0}, enc_v1 = {enc_v1}, enc_v2 = {enc_v2}, total = {agency.getTotal()}")
assert agency.getTotal() == 2

secret = 10 secret = 30 secret = 3
sk = 33, pk = 8
enc_v0 = (40, 33), enc_v1 = (16, 21), enc_v2 = (22, 10), total = 2


In [338]:
def make_more_votes():
    q = group.getQ()
    n = q - 1 # m is a vote and in mp = g**m %p it is an exponent so implicitly % q is applicated on it, so the sum cannot be higher than q
    trustees = [None] * n
    for i in range(n):
        trustees[i] = ExponentialElGamalTrustee(group)
    agency = ExponentialElGamalServer(group)
    for i in range(100):
        agency.reset()
        agency.generatePublicKey()
        pk = agency.getPublicKey()
        count = 0
        for j in range(n):
            vote = randint(0, 1)
            count += vote
            trustees[j].generateSecret()
            c1, c2 = trustees[j].vote(pk, vote)
            agency.receive(c1, c2)
        assert count == agency.getTotal(), f"{count} vs {agency.getTotal()}"

make_more_votes()

## Distributed Key Generation ElGamal

Publish a message decryptable by everyone (including outsiders of the group) only when all members of a group agrees to decrypt it.

In [339]:
class DistributedKeyGenerationElGamalMember:
    def __init__(self, group: Group):
        self.group = group
        self.generateSecret()
        self.generatePartialPublicKey()

    def __str__(self):
        return f""
    
    def generateSecret(self):
        q = self.group.getQ()
        self.xi = randint(0, q - 1)
    
    def generatePartialPublicKey(self):
        p, g, q, _ = self.group.getValues()
        xi = self.xi
        hi = pow(g, xi, p)
        self.hi = hi
        self.h = hi
    
    def receive_hi(self, hi: int):
        p = self.group.getP()
        h = self.h
        h = (h * hi) % p
        self.h = h
    
    def receive_di(self, di: int):
        p = self.group.getP()
        d = self.d
        d = (di * d) % p
        self.d = d
    
    def receive_enc_m(self, c1: int, c2: int):
        p = self.group.getP()
        self.c1 = c1
        self.c2 = c2
        xi = self.xi
        di = pow(c1, xi, p)
        self.di = di
        self.d = di

    def get_hi(self):
        return self.hi
    
    def get_di(self):
        return self.di
    
    def get_h(self):
        return self.h
    
    def get_d(self):
        return self.d

    def encrypt(self, m: int):
        p, g, q, _ = self.group.getValues()
        xi = self.xi
        h = self.h
        r = randint(0, q - 1)
        c1 = pow(g, r, p)
        c2 = ( pow(g, m, p) * pow(h, r, p) ) % p
        self.c1 = c1
        self.c2 = c2

        xi = self.xi
        di = pow(c1, xi, p)
        self.di = di
        self.d = di

        return c1, c2
    
    def decrypt(self):
        p, g, q, _ = self.group.getValues()
        d = self.d
        d_1 = self.group.getInverse(d)
        c2 = self.c2
        g_m = (c2 * d_1) % p
        return discrete_log(p, g_m, g)
    
member0 = DistributedKeyGenerationElGamalMember(group)
member1 = DistributedKeyGenerationElGamalMember(group)
member2 = DistributedKeyGenerationElGamalMember(group)

h0 = member0.get_hi()
h1 = member1.get_hi()
h2 = member2.get_hi()

member0.receive_hi(h1)
member0.receive_hi(h2)
member1.receive_hi(h0)
member1.receive_hi(h2)
member2.receive_hi(h0)
member2.receive_hi(h1)

assert member0.get_h() == member1.get_h() == member2.get_h(), f"{member0.get_h()} vs {member1.get_h()} vs {member2.get_h()}"

m = 14
c1, c2 = member0.encrypt(m) # 0 <= m < q because it's an exponent

member1.receive_enc_m(c1, c2)
member2.receive_enc_m(c1, c2)

d0 = member0.get_di()
d1 = member1.get_di()
d2 = member2.get_di()

member0.receive_di(d1)
member0.receive_di(d2)
member1.receive_di(d0)
member1.receive_di(d2)
member2.receive_di(d0)
member2.receive_di(d1)

assert member0.get_d() == member1.get_d() == member2.get_d(), f"{member0.get_d()} vs {member1.get_d()} vs {member2.get_d()}"

m0 = member0.decrypt()
m1 = member0.decrypt()
m2 = member0.decrypt()

assert m == m0, f"{m} vs {m0}"
assert m == m1, f"{m} vs {m1}"
assert m == m2, f"{m} vs {m2}"


In [340]:
def transcript_DistributedKeyGenerationElGamal():
    n = 100
    members = [None] * n
    for i in range(n):
        members[i] = DistributedKeyGenerationElGamalMember(group)
    
    q = group.getQ()
    for _ in range(10):
        r = randint(0, n - 1)

        for i in range(n):
            members[i].generatePartialPublicKey()

        for i in range(n):
            for j in range(n):
                if i != j:
                    members[i].receive_hi(members[j].get_hi())

        h = members[r].get_h()
        for i in range(n):
            assert members[i].get_h() == h, f"{members[i].get_h()} vs {h}"

        m = randint(0, q - 1)
        leader: DistributedKeyGenerationElGamalMember = members[r]
        c1, c2 = leader.encrypt(m)
        for i in range(n):
            if i == r:
                continue
            follower: DistributedKeyGenerationElGamalMember = members[i]
            follower.receive_enc_m(c1, c2)

        for i in range(n):
            for j in range(n):
                if i != j:
                    members[i].receive_di(members[j].get_di())

        d = members[r].get_d()
        for i in range(n):
            assert members[i].get_d() == d, f"{members[i].get_d()} vs {d}"

        
        for i in range(n):
            if i == r:
                continue
            follower: DistributedKeyGenerationElGamalMember = members[i]
            mp = follower.decrypt()
            assert m == mp, f"{m} vs {mp}"

transcript_DistributedKeyGenerationElGamal()

## Chaum Pederson

In [341]:
class ChaumPedersonProver:
    def __init__(self, group1: Group, group2: Group):
        assert group1.getP() == group2.getP()
        assert group1.getQ() == group2.getQ()
        self.group1 = group1
        self.group2 = group2
        self.generateSecrets()

    def __str__(self):
        return f""
    
    def generateSecrets(self):
        p, q = self.group1.getP(), self.group1.getQ()
        x = randint(0, q - 1)
        self.x = x
        g1 = self.group1.getG()
        g2 = self.group2.getG()
        self.h1 = pow(g1, x, p)
        self.h2 = pow(g2, x, p)

    def getPublicKeys(self):
        return self.h1, self.h2

    def getCommitment(self):
        p, q = self.group1.getP(), self.group1.getQ()
        g1 = self.group1.getG()
        g2 = self.group2.getG()
        r = randint(0, q - 1)
        self.r = r
        a1 = pow(g1, r, p)
        a2 = pow(g2, r, p)
        return a1, a2

    def getProof(self, e: int):
        q = self.group1.getQ()
        r = self.r
        x = self.x
        f = (r + e * x) % q # Modulo q it is an exponent
        return f

class ChaumPedersonVerifier:
    def __init__(self, group1: Group, group2: Group):
        assert group1.getP() == group2.getP()
        assert group1.getQ() == group2.getQ()
        self.group1 = group1
        self.group2 = group2

    def receivePublicKeys(self, h1: int, h2: int):
        self.h1 = h1
        self.h2 = h2

    def getChallenge(self, a1: int, a2: int):
        self.a1 = a1
        self.a2 = a2
        e = randint(0, 2**n)
        self.e = e
        return e
    
    def checkProof(self, f: int):
        p = self.group1.getP()
        g1 = self.group1.getG()
        g2 = self.group2.getG()
        h1 = self.h1
        h2 = self.h2
        a1 = self.a1
        a2 = self.a2
        e = self.e
        statement1 = pow(g1, f, p) == (a1 * pow(h1, e, p)) % p
        statement2 = pow(g2, f, p) == (a2 * pow(h2, e, p)) % p
        return statement1 and statement2
    
group2 = group.getRandomGroupFromSameOrder()
prover = ChaumPedersonProver(group, group2)
malicious = ChaumPedersonProver(group, group2)

# Both malicious and prover must have a different secret else the algorithm is broken
while malicious.x == prover.x:
    malicious.generateSecrets()

verifier = ChaumPedersonVerifier(group, group2)
h1, h2 = prover.getPublicKeys()
verifier.receivePublicKeys(h1, h2)

a1, a2 = prover.getCommitment()
e = verifier.getChallenge(a1, a2)
f = prover.getProof(e)
assert verifier.checkProof(f)

result = True
for _ in range(10):
    a1, a2 = malicious.getCommitment()
    e = verifier.getChallenge(a1, a2)
    f = malicious.getProof(e)
    result = result and verifier.checkProof(f)
assert not result


In [342]:
def transcript_ChaumPederson(prover: ChaumPedersonProver, malicious: ChaumPedersonProver, verifier: ChaumPedersonVerifier):
    for _ in range(100):
        prover.generateSecrets()
        malicious.generateSecrets()


        # Both malicious and prover must have a different secret else the algorithm is broken
        while malicious.x == prover.x:
            malicious.generateSecrets()

        h1, h2 = prover.getPublicKeys()
        verifier.receivePublicKeys(h1, h2)

        a1, a2 = prover.getCommitment()
        e = verifier.getChallenge(a1, a2)
        f = prover.getProof(e)
        assert verifier.checkProof(f)

        result = True
        for _ in range(10000):
            a1, a2 = malicious.getCommitment()
            e = verifier.getChallenge(a1, a2)
            f = malicious.getProof(e)
            result = result and verifier.checkProof(f)
        assert not result

transcript_ChaumPederson(prover, malicious, verifier)


## Non Interactive Zero Knowledge Proof

In [ ]:
def myhash(g: int, h: int, a: int):
    return abs(hash(str(g)) * hash(str(h)) * hash(str(a)))

class NISchnorrProver:
    def __init__(self, group: Group):
        self.group = group
        self.generateSecret()

    def generateSecret(self):
        p, g, q, _ = self.group.getValues()
        x = randint(0, q - 1) # Modulo q as the secret is meant to be an exponent
        self.x = x
        self.h = pow(g, x, p)
    
    def getPublicKey(self):
        return self.h

    def getCommitmentAndChallenge(self):
        p, g, q, _ = self.group.getValues()
        r = randint(0, q - 1)
        a = pow(g, r, p)
        h = self.h
        e = myhash(g, h, a) % p
        x = self.x
        f = (r + e * x) % q # Modulo q it is an exponent
        return a, e, f
    
class NISchnorrVerifier:
    def __init__(self, group: Group):
        self.group = group

    def receivePublicKey(self, h: int):
        self.h = h
    
    def checkProof(self, a: int, e: int, f: int):
        p, g, q, _ = self.group.getValues()
        h = self.h
        return pow(g, f, p) == (a * pow(h, e, p)) % p
    
prover = NISchnorrProver(group)
malicious = NISchnorrProver(group)
verifier = NISchnorrVerifier(group)

In [344]:
def transcript_NISchnorr(prover: NISchnorrProver, malicious: NISchnorrProver, verifier: NISchnorrVerifier):
    for _ in range(100):
        prover.generateSecret()
        malicious.generateSecret()
        while prover.x == malicious.x:
            malicious.generateSecret()
        verifier.receivePublicKey(prover.getPublicKey())

        for _ in range(10):
            a, e, f = prover.getCommitmentAndChallenge()
            assert verifier.checkProof(a, e, f)
            
            result = True
            for _ in range(100):
                a, e, f = malicious.getCommitmentAndChallenge()
                result = result and verifier.checkProof(a, e, f)
            assert not result

transcript_NISchnorr(prover, malicious, verifier)